In [1]:
!nvidia-smi

Sat Nov 15 08:48:27 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


In [3]:
# Load Model and tokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct", 
    device_map = "cuda", 
    torch_dtype = "auto", 
    trust_remote_code = True, 
)

# Creata a pipeline

generator = pipeline(
    "text-generation", 
    model = model, 
    tokenizer = tokenizer,
    return_full_text = False, 
    max_new_tokens = 50, 
    do_sample = False
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Device set to use cuda
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [4]:
prompt = "Write an email apologizing to sarah for the tragic gardening mishap. Explain how it happened."

output = generator(prompt, use_cache = False)

print(output[0]["generated_text"])



Dear Sarah,

I am deeply sorry for the unfortunate incident that occurred in my garden. I understand that it must have been quite distressing for you to witness the damage caused to your beloved plants.




In [5]:
print(model)

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLUActivation()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out_fea

In [6]:
prompt = 'The Capital of france is'

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# Tokenize the input prompt:
input_ids = input_ids.to("cuda")

# Get the output of the model before the lm_head
model_output = model.model(input_ids, use_cache=False)

# Get the output of the lm head
lm_head_output = model.lm_head(model_output[0])

In [7]:
model_output

BaseModelOutputWithPast(last_hidden_state=tensor([[[-0.3047,  1.1953,  0.2988,  ..., -0.3008,  0.6758,  0.1406],
         [-0.6367,  1.0234, -0.4805,  ...,  0.8125, -0.2461, -0.3164],
         [-0.3594,  1.1250,  1.5156,  ...,  0.0854,  0.1240,  0.3203],
         [-0.0530,  0.7188,  0.2617,  ...,  0.8359, -0.6641,  0.7266],
         [-0.6641,  1.1875,  0.8750,  ...,  0.0488,  0.1855,  0.0854],
         [-1.1016,  0.5742,  0.0913,  ...,  0.3906, -0.0396,  0.2715]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<MulBackward0>), past_key_values=None, hidden_states=None, attentions=None)

In [8]:
lm_head_output

tensor([[[24.7500, 24.8750, 22.7500,  ..., 19.0000, 19.0000, 19.0000],
         [26.6250, 28.6250, 26.0000,  ..., 21.7500, 21.7500, 21.7500],
         [33.0000, 31.5000, 32.5000,  ..., 26.6250, 26.6250, 26.6250],
         [14.6250, 12.6875, 15.1250,  ..., 10.0625, 10.0625, 10.0625],
         [32.2500, 31.8750, 34.0000,  ..., 27.5000, 27.5000, 27.5000],
         [27.3750, 27.8750, 26.2500,  ..., 20.7500, 20.7500, 20.7500]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<UnsafeViewBackward0>)

In [9]:
token_id = lm_head_output[0, -1].argmax(-1)
tokenizer.decode(token_id)

'par'

In [11]:
prompt = "Write a very long email apologizing to Sarah for the tragic gardeningmishap. Explain how it happened."
# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to("cuda")

In [21]:
%%timeit -n 1
# Generate the text
generation_output = model.generate(
input_ids=input_ids,
max_new_tokens=100,
use_cache=False
)

33.3 s ± 275 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
